In [1]:
import pandas as pd

In [2]:
from fast_pandas import FastPandas

In [3]:
df = pd.DataFrame({"a": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]})

In [5]:
FastPandas(df)["a"].abs().approx_count_distinct().df()

,approx_count_distinct(abs(a))
0,10
